# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import warnings
%load_ext memory_profiler

warnings.filterwarnings("ignore")

if not os.path.exists("parrot.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/parrot.bmp --no-check-certificate
parrot = cv2.imread('parrot.bmp', cv2.COLOR_BGR2GRAY)

A_invert = np.array([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[-3,  3,  0,  0, -2, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[2, -2,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
[0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
[0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
[0,  0,  0,  0,  0,  0,  0,  0, -3,  3,  0,  0, -2, -1,  0,  0],
[0,  0,  0,  0,  0,  0,  0,  0,  2, -2,  0,  0,  1,  1,  0,  0],
[-3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0,  0,  0,  0,  0],
[0,  0,  0,  0, -3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0],
[9, -9, -9,  9,  6,  3, -6, -3,  6, -6,  3, -3,  4,  2,  2,  1],
[-6,  6,  6, -6, -3, -3,  3,  3, -4,  4, -2,  2, -2, -2, -1, -1],
[2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0],
[0,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0],
[-6,  6,  6, -6, -4, -2,  4,  2, -3,  3, -3,  3, -2, -1, -2, -1],
[4, -4, -4,  4,  2,  2, -2, -2,  2, -2,  2, -2,  1,  1,  1,  1],
])

In [ ]:
def bilinear_interpolation(image, scale_x, scale_y):
    X, Y = image.shape
    new_X, new_Y = int(np.round(X*scale_x)), int(np.round(Y*scale_y))
    new_image = np.zeros((new_X, new_Y))
    for i in range(new_X):
        for j in range(new_Y):
            i1 = int(np.floor(i / scale_x))
            j1 = int(np.floor(j / scale_y))
            i2 = i1 + 1
            j2 = j1 + 1
            if i2 > X - 1:
                i1 = X - 1
                i2 = i1 - 1
            if j2 > Y - 1:
                j1 = Y - 1
                j2 = j1 - 1
            ipom = i / scale_x - i1
            jpom = j / scale_y - j1
            A = image[i1][j1]
            B = image[i2][j1]
            C = image[i2][j2]
            D = image[i1][j2]
            ABCD = A * (1 - ipom) * (1 - jpom) + B * ipom * (1 - jpom) + C * ipom * jpom + D * jpom * (1 - ipom)
            new_image[i][j] = int(np.round(ABCD))
    plt.figure(figsize=(new_image.shape[0]/100,new_image.shape[1]/100), dpi=200)
    plt.imshow(new_image, cmap ="gray")
    plt.axis('off')
    plt.show()

In [ ]:
def bicubic_interpolation(image, scale_x, scale_y):
    X, Y = image.shape
    new_X, new_Y = int(np.round(X * scale_x)), int(np.round(Y * scale_y))
    new_image = np.zeros((new_X, new_Y))

    for i in range(0, new_X):
        for j in range(0, new_Y):
            i1 = int(np.floor(i / scale_x))
            j1 = int(np.floor(j / scale_y))
            i2 = i1 + 1
            j2 = j1 + 1

            if i2 > X - 2:
                i1 = X - 2
                i2 = i1 - 2

            if j2 > Y - 2:
                j1 = Y - 2
                j2 = j1 - 2


            ipom = i / scale_x - i1
            jpom = j / scale_y - j1

            A = image[i1][j1]
            B = image[i2][j1]
            C = image[i2][j2]
            D = image[i1][j2]

            Ax = (image[i1 + 1][j1] - image[i1 - 1][j1])/2
            Bx = (image[i2 + 1][j1] - image[i2 - 1][j1])/2
            Cx = (image[i2 + 1][j2] - image[i2 - 1][j2])/2
            Dx = (image[i1 + 1][j2] - image[i1 - 1][j2])/2

            Ay = (image[i1][j1 + 1] - image[i1][j1 - 1])/2
            By = (image[i2][j1 + 1] - image[i2][j1 - 1])/2
            Cy = (image[i2][j2 + 1] - image[i2][j2 - 1])/2
            Dy = (image[i1][j2 + 1] - image[i1][j2 - 1])/2

            Axy = (image[i1 + 1][j1 + 1] - image[i1 - 1][j1] - image[i1][j1 - 1] + image[i1][j1])/4
            Bxy = (image[i2 + 1][j1 + 1] - image[i2 - 1][j1] - image[i2][j1 - 1] + image[i2][j1])/4
            Cxy = (image[i2 + 1][j2 + 1] - image[i2 - 1][j2] - image[i2][j2 - 1] + image[i2][j2])/4
            Dxy = (image[i1 + 1][j2 + 1] - image[i1 - 1][j2] - image[i1][j2 - 1] + image[i1][j2])/4

            x = np.transpose(np.array([A,B,D,C,Ax,Bx,Dx,Cx,Ay,By,Dy,Cy,Axy,Bxy,Dxy,Cxy]))
            a = np.dot(A_invert,x)
            a = a.reshape((4,4),order='F')
            ABCD = 0

            for iq in range(4):
                for jq in range(4):
                    ABCD += a[iq][jq] * (ipom ** iq) * (jpom ** jq)
            new_image[i][j] = int(np.round(ABCD))

    plt.figure(figsize=(new_image.shape[0] / 100, new_image.shape[1] / 100), dpi=200)
    plt.imshow(new_image, cmap="gray")
    plt.axis('off')
    plt.show()

In [ ]:
%%capture
t1 = %timeit -o bilinear_interpolation(parrot, 2.5, 2.5)
m1 = %mprun -o bilinear_interpolation(parrot, 2.5, 2.5)
t2 = %timeit -o bicubic_interpolation(parrot, 2.5, 2.5)
m2 = %mprun -o bicubic_interpolation(parrot, 2.5, 2.5)

In [ ]:
bilinear_interpolation(parrot, 2.5, 2.5)
print('Interpolacja dwuliniowa - czas niezbędny do obliczeń:',t1)
print('Pamięć zużywana przez interpolację dwuliniową:', m1)
bicubic_interpolation(parrot, 2.5, 2.5)
print('Interpolacja dwusześcienna - czas niezbędny do obliczeń',t2)
print('Pamięć zużywana przez interpolację dwusześcienną:', m2)

W powyższej implementacji interpolacja dwuliniowa jest praktycznie około 400 razy szybsza od interpolacji dwusześciennej. Zużycie pamięci jest za to na podobnym poziomie. W implementacji interpolacji dwusześciennej niestety zapodział się błąd z uwagi na występujące widoczne granice między obszarami interpolacji oraz zaciemnianie obrazu przez funkcję gdy obecne są warunki na wyjście poza zakres.